In [38]:
import requests
import time

# commander_input = "sheoldred the apocalypse"
commander_input = "Queza, Augur of Agonies"
# num_ramp = 10
# num_draw = 10
# num_removal = 10
num_ramp = 3
num_draw = 20
num_removal = 7

HEADERS = {
    "User-Agent": "deck-builder-bot/1.0",
    "Accept": "application/json"
}

def get_commander(name):
    url = "https://api.scryfall.com/cards/named"
    params = {
        "exact": name
    }

    r = requests.get(url, params=params, headers=HEADERS)
    r.raise_for_status()
    return r.json()


def fetch_all_cards(query):
    url = "https://api.scryfall.com/cards/search"
    params = {"q": query}
    print(query)
    cards = []
    card_index = 0
    while True:
        
        r = requests.get(url, params=params, headers=HEADERS)
        r.raise_for_status()
        data = r.json()
        card_index += len(data["data"]) 
        total_cards = data["total_cards"]
        percentage = float(card_index)/data["total_cards"] * 100
        print(f"current index {card_index}, total cards {total_cards}, percentage {percentage}")

        cards.extend(data["data"])

        if not data.get("has_more"):
            break

        # IMPORTANT: next_page already includes query params
        url = data["next_page"]
        params = None
        time.sleep(0.2) # dont want to get rate limited
    return cards


In [39]:
commander = get_commander(commander_input)

In [40]:
color_identity = commander["color_identity"]
print(commander['name'], color_identity)  # e.g. ['G', 'U', 'B', 'W']
def color_identity_query(ci):
    if not ci:
        return "ci=0"   # colorless commanders (Kozilek, etc.)
    return f"ci<={''.join(ci)}"
cid_string = color_identity_query(color_identity)
print(cid_string)

Queza, Augur of Agonies ['B', 'U', 'W']
ci<=BUW


In [41]:
QUERIES = {
    "ramp" : f"usd<5 function:ramp {cid_string} cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow",
    "draw" : f"usd<5 function:draw {cid_string} cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow",
    "removal" : f"usd<5 function:removal {cid_string} -type:land -set:UNF f:commander -is:gamechanger -o:snow"
}

ramp_cards    = fetch_all_cards(QUERIES["ramp"])
draw_cards    = fetch_all_cards(QUERIES["draw"])
removal_cards = fetch_all_cards(QUERIES["removal"])

usd<5 function:ramp ci<=BUW cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow
current index 175, total cards 449, percentage 38.97550111358575
current index 350, total cards 449, percentage 77.9510022271715
current index 449, total cards 449, percentage 100.0
usd<5 function:draw ci<=BUW cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow
current index 175, total cards 1747, percentage 10.01717229536348
current index 350, total cards 1747, percentage 20.03434459072696
current index 525, total cards 1747, percentage 30.05151688609044
current index 700, total cards 1747, percentage 40.06868918145392
current index 875, total cards 1747, percentage 50.085861476817406
current index 1050, total cards 1747, percentage 60.10303377218088
current index 1225, total cards 1747, percentage 70.12020606754436
current index 1400, total cards 1747, percentage 80.13737836290784
current index 1575, total cards 1747, percentage 90.15455065827133
current index 1747, total cards 1747, p

In [42]:
import random

def pick_random(cards, n):
    if len(cards) < n:
        raise ValueError(f"Not enough cards to sample {n}")
    return random.sample(cards, n)



chosen_ramp    = pick_random(ramp_cards, num_ramp)
chosen_draw    = pick_random(draw_cards, num_draw)
chosen_removal = pick_random(removal_cards, num_removal)

In [43]:
COLOR_TO_BASIC = {
    "W": "Plains",
    "U": "Island",
    "B": "Swamp",
    "R": "Mountain",
    "G": "Forest"
}

def get_basic_lands(color_identity, total_basics=16):
    if not color_identity:
        return ["Wastes"] * total_basics

    basics = []
    per_color = total_basics // len(color_identity)
    remainder = total_basics % len(color_identity)

    for c in color_identity:
        basics.extend([COLOR_TO_BASIC[c]] * per_color)

    for i in range(remainder):
        basics.append(COLOR_TO_BASIC[color_identity[i]])

    return basics

def multicolor_land_query(ci):
    return (
        f"type:land -type:basic legal:commander -is:gamechanger "
        f"ci<={''.join(ci)} "
        f"o:/add .*[{''.join(ci)}].*[{''.join(ci)}]/"
    )


FETCH_QUERY = (
    "type:land legal:commander "
    "(o:search or o:sacrifice) -is:gamechanger "
    "-o:add"
)

multicolor_lands = fetch_all_cards(multicolor_land_query(color_identity))
fetch_lands = fetch_all_cards(FETCH_QUERY)

type:land -type:basic legal:commander -is:gamechanger ci<=BUW o:/add .*[BUW].*[BUW]/
current index 167, total cards 167, percentage 100.0
type:land legal:commander (o:search or o:sacrifice) -is:gamechanger -o:add
current index 31, total cards 31, percentage 100.0


In [44]:
import random
print('getting land base')
def pick(cards, n):
    return random.sample(cards, min(n, len(cards)))


chosen_fetches    = pick(fetch_lands, 10)

land_base = []
basic_lands = []
chosen_multicolor = []
if len(color_identity) == 1:
    print('getting only basics')
    basic_lands       = get_basic_lands(color_identity, 28)
elif len(color_identity) == 2:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 16)
    chosen_multicolor = pick(multicolor_lands, 12)
elif len(color_identity) == 3:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 8)
    chosen_multicolor = pick(multicolor_lands, 20)
elif len(color_identity) >= 4:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 4)
    chosen_multicolor = pick(multicolor_lands, 24)

# Basics (names only)
for b in basic_lands:
    land_base.append({"name": b})

# Multicolor lands
land_base.extend(chosen_multicolor)

# Fetch lands
land_base.extend(chosen_fetches)

getting land base
getting multi lands and basics


In [45]:
SOL_RING = "Sol Ring"

In [46]:
def random_filler_query(ci):
    ci_filter = f"ci<={''.join(ci)}" if ci else "ci=0"

    return (
        f"legal:commander "
        f"{ci_filter} "
        f"-type:land "
        f"-is:funny "
        f"-set:UNF "
        f"-o:snow "
        f"-type:conspiracy "
        f"-type:emblem "
        f"-name:\"Sol Ring\" "
        f"usd<=1"
    )

print("fetching filler cards")
filler_cards = fetch_all_cards(random_filler_query(color_identity))
random_cards = random.sample(filler_cards, min(30, len(filler_cards)))


fetching filler cards
legal:commander ci<=BUW -type:land -is:funny -set:UNF -o:snow -type:conspiracy -type:emblem -name:"Sol Ring" usd<=1
current index 175, total cards 14094, percentage 1.2416631190577552
current index 350, total cards 14094, percentage 2.4833262381155103
current index 525, total cards 14094, percentage 3.7249893571732655
current index 700, total cards 14094, percentage 4.966652476231021
current index 875, total cards 14094, percentage 6.208315595288775
current index 1050, total cards 14094, percentage 7.449978714346531
current index 1225, total cards 14094, percentage 8.691641833404287
current index 1400, total cards 14094, percentage 9.933304952462041
current index 1575, total cards 14094, percentage 11.174968071519796
current index 1750, total cards 14094, percentage 12.41663119057755
current index 1925, total cards 14094, percentage 13.658294309635306
current index 2100, total cards 14094, percentage 14.899957428693062
current index 2275, total cards 14094, percen

In [47]:
deck = []
deck.append(commander)
# Core spells
deck.extend(chosen_ramp)
deck.extend(chosen_draw)
deck.extend(chosen_removal)

# Lands
deck.extend(land_base)

# Random filler
deck.extend(random_cards)

# Sol Ring
deck.append({"name": SOL_RING})

In [48]:
from collections import Counter

counts = Counter(c["name"] for c in deck)

for name, count in sorted(counts.items()):
    print(f"{count} {name}")

1 Abstruse Interference
1 Adaptive Shimmerer
1 Aesthir Glider
1 Alley Assailant
1 Amass the Components
1 Arcane Flight
1 Arcane Sanctum
1 Arid Mesa
1 Barrin's Codex
1 Basal Sliver
1 Battlefield Thaumaturge
1 Boneclad Necromancer
1 Boreal Shelf
1 Brokers Hideout
1 Cabal Pit
1 Cabal Trainee
1 Cabaretti Courtyard
1 Cactus Preserve
1 Call the Cavalry
1 Chant of Vitu-Ghazi
1 Circle of Affliction
1 Clairvoyance
1 Clan Crafter
1 Cloudpost
1 Combat Tutorial
1 Crystalline Crawler
1 Dark Depths
1 Darkblade Agent
1 Devouring Deep
1 Devouring Light
1 Douse in Gloom
1 Entrapment Maneuver
1 Exultant Cultist
1 Eye of Ugin
1 Fetid Heath
1 Fortified Beachhead
1 Goldmire Bridge
1 Great Hall of the Citadel
1 Gurgling Anointer
1 Inspiring Roar
3 Island
1 Katara, Bending Prodigy
1 Lay Down Arms
1 Leaden Fists
1 Lilypad Village
1 Mage-Ring Network
1 Malcolm, Alluring Scoundrel
1 Marsh Flats
1 Messenger Hawk
1 Mischievous Catgeist // Catlike Curiosity
1 Mistvault Bridge
1 Mystic Gate
1 Nykthos, Shrine to Nyx